# 5.1 Ejemplo 2. Cálculo de optimización geométrica para celda de silicio. *Relax*


Una de las herramientas básicas más útiles para realizar cálculos electrónicos, es la relajación geométrica de una molécula o estructura cristalina. El sistema con el que se trabaje, debe de encontrarse, en la mayoría de los casos, en un mínimo energético, en donde los iones estén sometidos al mínimo de fuerza y/o esfurzo. El ejemplo que se cubre en este notebook muestra la manera de realizar este cálculo, asumiendo que previamente se obtuvieron los parámetros de convergencia *ecutwfn* y *k-points* para el sistema del cristal de silicio.
En la figura siguiente se muestra la visualización con *XCrySDen*, tanto de la celda primitiva del silicio, como una repetición de $2X2X2$ celdas primitivas con 4 átomos coloreados para una visualización de la estructura tipo diamante del silicio, en la que cada átomo tiene 4 enlace covalentes con sus 4 primeros vecinos. En este ejercicio se muestran algunas otras propiedades que se pueden obtener mediante el cálculo de optimización geométrica 
(*"relax"*).


<img src="Figs/Si1cell2atom.png" style="width: 300px">
<img src="Figs/Si4cell4atom.png" style="width: 300px">

**Figura**. Celda primitiva y 4 celdas primitivas de Si. Representación utilizando el código **XCrySDen**.

## I. cálculo de optimización geométrica *relax*.

En QE, utilizar la opción "*relax*" en *calculation*, permite obtener la ciclos de optimización de un archivo -cada uno consistente en un ciclo *scf* como el del [notebook anterior](SinglePoint-convergence-plotting.ipynb), pero entre cada uno de estos ciclos, se calculan las fuerzas entre átomos y el tensor de esfuerzo del sistema, lo cual, mediante un algoritmo a elegir -**bfgs o damp**- lleva a una minimización de las fuerzas y esfuerzos (presión), idealmente por debajo de un umbral, también parámetro de entrada o definido por *default*. 

### Archivo de entrada, *Input file*: "name.in"

Entre las variables para la relajación geométrica, están la relajación de los iones -*relax*- y la relajación de la estructura -*vc-relax*-. A continuación se muestra un ejemplo de archivo de entrada para cálculos tipo *vc-relax*, llamado [*Si_relax.in*](Si_relax.in):

In [39]:
%more Si_relax.in 

### Si_relax:

En el siguiente cuadro también se aprecia el archivo Si_relax.in:


$
&control
    calculation = 'vc-relax'
    restart_mode='from_scratch',
    prefix='Si_relax',
    tstress = .true.
    tprnfor = .true.
    pseudo_dir = '/qe-6.3/pseudo',
    outdir='/qe-6.3/tempdir/'
 /
 &system
    ibrav=  2, celldm(1) =9.5, nat=  2, ntyp= 1,
    ecutwfc =40.0,
 /
 &electrons
    diagonalization='david'
    mixing_mode = 'plain'
    mixing_beta = 0.7
    conv_thr =  1.0d-8
 /
 &IONS
 ion_dynamics = 'bfgs'
 /
 &CELL
 cell_dynamics='bfgs'
 cell_dofree='all'
 /
ATOMIC_SPECIES
 Si  28.086  Si.pz-vbc.UPF
ATOMIC_POSITIONS
 Si 0.00 0.00 0.00
 Si 0.26 0.24 0.27
K_POINTS {automatic}
4 4 4   0 0 0
$

En este archivo, asumimos -quedando como **Ejercicio 2**, que la convergencia de *ecutwfc* y la malla de *k-points* está convergida, utilizando **40 Ry** y **4X4X4** kpoints respectivamente. La instrucción *cell_dofree*, en este caso "all", indica que la celda, tanto parámetros de red como ángulos, están libres para optimizarse para la relajación de celda. Del mismo modo, en este caso no hay restricciones en las posiciones de los 2 átomos de Si
La celda ideal de Si, tiene *celldm(1)*=10.5 y las coordenadas del segundo átomo de Si en 0.25 0.25 0.25. Las pequeñas variaciones se introdujeron, precisamente para propositos de realizar una optimización geométrica. 

Para realizar el cálculo con cuatro procesadores en paralelo utilizamos el siguiente comando:

In [103]:
!mpirun --np 4 /qe-6.3/bin/pw.x < Si_relax.in > Si_relax.out

Note: The following floating-point exceptions are signalling: IEEE_DENORMALNote: The following floating-point exceptions are signalling: IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_DENORMAL



El archivo de salida es **Si_relax.out**, muestra una convergencia tanto de la celda como de la posición atómica, realizada en 21.24 segundos utilizando 8 pasos de relajación **BFGS** (BFGS es un algoritmo quasi-newotoniano nombrado por con las iniciales de los científicos Charles George Broyden, Roger Fletcher, Donald Goldfarb y David Shanno). 

In [110]:
%more Si_relax.out

También se puede accesar a una versión en formato de texto [Si_relax_texto](Si_relax.out), una versión pdf [Si_relax.pdf](Docs/Si_relax.out.pdf), o bien, buscar información específica con el comando *grep* en la terminal o en este notebook: Las energías totales del sistema después de cada optimización geométrica:

In [5]:
!grep "!    total energy " Si_relax.out 

!    total energy              =     -15.77996140 Ry
!    total energy              =     -15.83485167 Ry
!    total energy              =     -15.83813214 Ry
!    total energy              =     -15.83850825 Ry
!    total energy              =     -15.83861610 Ry
!    total energy              =     -15.83867859 Ry
!    total energy              =     -15.83869110 Ry
!    total energy              =     -15.83869214 Ry
!    total energy              =     -15.83904851 Ry


La modificación de las posiciones relativas de los dos átomos de Si de la celda primitiva, o el cambio de las fuerzas o estrés del sistema:

In [113]:
!grep "Si       0" Si_relax.out 

Si       0.004647307  -0.001546203   0.009046137
Si       0.266323212   0.259154525   0.274102350
Si       0.004415131  -0.000354305   0.008731179
Si       0.271395521   0.264068841   0.278688918
Si       0.004653198  -0.000120658   0.009309405
Si       0.273359057   0.266602042   0.279969800
Si       0.004958827  -0.000062866   0.009929819
Si       0.274183624   0.268046945   0.280203028
Si       0.005291912  -0.000029744   0.010585886
Si       0.274626979   0.269145824   0.280051556
Si       0.005394448  -0.000019504   0.010784379
Si       0.274572860   0.269285184   0.279833826
Si       0.005390412  -0.000011825   0.010776741
Si       0.274480955   0.269172415   0.279768262
Si       0.005390412  -0.000011825   0.010776741
Si       0.274480955   0.269172415   0.279768262


In [114]:
!grep "total   stress" Si_relax.out 

          total   stress  (Ry/bohr**3)                   (kbar)     P=  335.53
          total   stress  (Ry/bohr**3)                   (kbar)     P=   58.75
          total   stress  (Ry/bohr**3)                   (kbar)     P=   12.59
          total   stress  (Ry/bohr**3)                   (kbar)     P=   -0.73
          total   stress  (Ry/bohr**3)                   (kbar)     P=   -3.98
          total   stress  (Ry/bohr**3)                   (kbar)     P=   -3.27
          total   stress  (Ry/bohr**3)                   (kbar)     P=   -1.01
          total   stress  (Ry/bohr**3)                   (kbar)     P=   -0.08
          total   stress  (Ry/bohr**3)                   (kbar)     P=    0.20


In [115]:
!grep "atom    1 type  1   force =" Si_relax.out 

     atom    1 type  1   force =     0.04142378   -0.01251719    0.08126431
     atom    1 type  1   force =    -0.00293467    0.00875997   -0.00537003
     atom    1 type  1   force =     0.00123317    0.00087066    0.00301369
     atom    1 type  1   force =     0.00152741   -0.00018595    0.00298559
     atom    1 type  1   force =     0.00096204   -0.00017856    0.00180530
     atom    1 type  1   force =     0.00021161    0.00000279    0.00037747
     atom    1 type  1   force =    -0.00003321    0.00007977   -0.00006323
     atom    1 type  1   force =    -0.00002442    0.00004718   -0.00004243
     atom    1 type  1   force =    -0.00002435    0.00004714   -0.00004248


En las tres series de datos obtenidos con el comando *grep*, se observa que la energía total, el estrés del sistema y la fuerza que actúa sobre el atomo 1, disminuyen. 

El volumen de la celda, aumenta y la configuración inicial y final de átomos y parámetros de red son: 

In [116]:
!grep  "new unit-cell volume =" Si_relax.out 

     new unit-cell volume =    253.05286 a.u.^3 (    37.49856 Ang^3 )
     new unit-cell volume =    263.80400 a.u.^3 (    39.09172 Ang^3 )
     new unit-cell volume =    267.44031 a.u.^3 (    39.63057 Ang^3 )
     new unit-cell volume =    268.37791 a.u.^3 (    39.76950 Ang^3 )
     new unit-cell volume =    268.17917 a.u.^3 (    39.74005 Ang^3 )
     new unit-cell volume =    267.54031 a.u.^3 (    39.64538 Ang^3 )
     new unit-cell volume =    267.27687 a.u.^3 (    39.60635 Ang^3 )
     new unit-cell volume =    267.27687 a.u.^3 (    39.60635 Ang^3 )


In [117]:
!grep -3 " a(1) = (" Si_relax.out


     celldm(4)=   0.000000  celldm(5)=   0.000000  celldm(6)=   0.000000

     crystal axes: (cart. coord. in units of alat)
               a(1) = (  -0.500000   0.000000   0.500000 )  
               a(2) = (   0.000000   0.500000   0.500000 )  
               a(3) = (  -0.500000   0.500000   0.000000 )  

--
     celldm(4)=   0.000000  celldm(5)=   0.000000  celldm(6)=   0.000000

     crystal axes: (cart. coord. in units of alat)
               a(1) = (  -0.538223  -0.000010   0.538070 )  
               a(2) = (   0.000010   0.538280   0.538062 )  
               a(3) = (  -0.538178   0.538215   0.000023 )  



In [14]:
!grep -2 "ATOMIC_POSITIONS" Si_relax.in

ATOMIC_SPECIES
 Si  28.086  Si.pz-vbc.UPF
ATOMIC_POSITIONS
 Si 0.00 0.00 0.00
 Si 0.26 0.25 0.27


In [15]:
!grep -3 "End final coordinates" Si_relax.out

ATOMIC_POSITIONS (alat)
Si       0.005390412  -0.000011825   0.010776741
Si       0.274480955   0.269172415   0.279768262
End final coordinates





Para obtener el valor de $a_0$ en angstroms, hay que convertir alguno de los parámetros de la red primitiva -que está en Bohrs, al parametro de red cúbica $a_0$ en angstroms. 

In [4]:
a0=(((0.5382**2)*2)**(1/2))*2*0.7071*9.5*0.52917
print(a0)

5.411134692471809


En comparación con el valor experimental de $5.41\overset{\circ}{A}$, significa un 0.35 % de error.

El siguiente video, realizado al leer el archivo de salida Si_relax.out con *XCrySDen*, y desplegar las fuerzas sobre los átomos, permite visualizar cómo disminuyen rapidamente. Sin embargo, el cambio del tamaño de la celda, de 5.0 a $5.382 \overset{\circ}{A}$ es indistinguible [VideoRelaxSi](Docs/SiRelax.mp4)


<video controls src="Docs/SiRelax.mp4" style="width: 500px"/>

Como **ejercicio 3**, queda realizar gráficas para mostrar el cambio de la energía y la presión en la celda conforme pasan los ciclos BFGS.  

El siguiente notebook se realiza un ejercicio completo para cálcular [la estructura de bandas del silicio.](band-structre_and_DOS_Si.ipynb)

[Ir al notebook anterior.](SinglePoint-convergence-plotting.ipynb)
